In [1]:
from pymoo.operators.sampling.rnd import BinaryRandomSampling
from pymoo.operators.mutation.bitflip import BitflipMutation
from pymoo.operators.crossover.hux import HUX
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize

from generic_optimizer import GenericOptimizer, prepare_splits
from novel_sampling import BiasedBinarySampling, over_sample

from generic_optimizer import *

from pymoo.core.mutation import Mutation

from scipy.stats import ranksums

from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
import os
import re

from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch
import torch.nn as nn

from sklearn.preprocessing import LabelEncoder

In [ ]:
class CustomDataset(Dataset):
	def __init__(self, x_train, y_train):
		self.x_train = x_train
		self.y_train = y_train
	def __len__(self):
		return self.x_train.shape[0]
	def __getitem__(self, ind):
		x = self.x_train[ind]
		y = self.y_train[ind]
		return x, y

class CustomMutation(Mutation):
	
	device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
	num_synthetic_examples = 500000
	train_epochs = 50
	batch_size = 215
	prediction_thresh = 0.8
	primary_objective = GenericOptimizer.calculate_overall_error
	secondary_objectives = [
		[GenericOptimizer.calculate_num_examples],
		[GenericOptimizer.calculate_class0_error],
		[GenericOptimizer.calculate_class0_inverse_f1],
		[GenericOptimizer.calculate_class0_inverse_precision],
		[GenericOptimizer.calculate_class0_inverse_recall],
		[GenericOptimizer.calculate_class1_error],
		[GenericOptimizer.calculate_class1_inverse_f1],
		[GenericOptimizer.calculate_class1_inverse_precision],
		[GenericOptimizer.calculate_class1_inverse_recall],
		[GenericOptimizer.calculate_overall_inverse_f1],
		[GenericOptimizer.calculate_overall_inverse_precision],
		[GenericOptimizer.calculate_overall_inverse_recall],
		[GenericOptimizer.calculate_class0_inverse_precision, GenericOptimizer.calculate_class1_inverse_precision],
		[GenericOptimizer.calculate_class0_inverse_recall, GenericOptimizer.calculate_class1_inverse_recall],
		[GenericOptimizer.calculate_class0_inverse_f1, GenericOptimizer.calculate_class1_inverse_f1],
		[GenericOptimizer.calculate_class0_error, GenericOptimizer.calculate_class1_error],
	]

	def __init__(self, x_train, y_train, x_validation, y_validation):
		super().__init__()

		synthesized_x, synthesized_y = CustomMutation.create_training_data(x_train, y_train, x_validation, y_validation)
		self.model = CustomMutation.train_mutation(synthesized_x, synthesized_y)

	def _do(self, problem, X, **kwargs):

		int_x = np.array(X, dtype=np.float32)
		dataset = CustomDataset(int_x, int_x)
		loader = DataLoader(dataset, batch_size=X.shape[0], shuffle=False)

		self.model.eval()
		with torch.no_grad():
			for data, _ in loader:
				data = data.to(CustomMutation.device)
				outputs = self.model(data)
				predictions = (outputs > CustomMutation.prediction_thresh).bool()

		return np.array(predictions)

	@classmethod
	def train_mutation(cls, x_train, y_train):
		train_set = CustomDataset(x_train, y_train)
		input_dim = x_train.shape[1]
		batch_size = cls.batch_size
		train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

		device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

		class MLP(nn.Module):
			def __init__(self):
				super(MLP, self).__init__()
				self.linear1 = nn.Linear(input_dim, input_dim//2)
				self.relu1 = nn.ReLU()
				self.linear2 = nn.Linear(input_dim//2, input_dim//3)
				self.relu2 = nn.ReLU()
				self.linear3 = nn.Linear(input_dim//3, input_dim)

			def forward(self, x):
				x = self.linear1(x)
				x = self.relu1(x)
				x = self.linear2(x)
				x = self.relu2(x)
				x = self.linear3(x)
				return x
			
		model = MLP().to(device)
		optimizer = torch.optim.Adam(model.parameters(), lr=0.000001)
		criterion = nn.BCEWithLogitsLoss()

		model.train()
		for epoch in range(cls.train_epochs):
			losses = []
			for batch_num, input_data in enumerate(train_loader):
				optimizer.zero_grad()
				x, y = input_data
				x, y = x.to(device).float(), y.to(device)

				output = model(x)
				loss = criterion(output, y)
				loss.backward()
				losses.append(loss.item())
				optimizer.step()

		return model
	
	@classmethod
	def create_training_data(cls, x_train, y_train, x_validation, y_validation):
		
		# Execute optimization and extract the final populations
		populations = Parallel(n_jobs=-1)(delayed(cls.execute_training_data_gen)((objectives, x_train, y_train, x_validation, y_validation)) for objectives in cls.secondary_objectives)
		
		# Aggregate all populations into single list containing every unique instance
		all_instances = []
		for population in populations:
			for individual in population:
				individual = list(individual)
				if individual not in all_instances:
					all_instances.append(individual)
		all_instances = np.array(all_instances)

		# Create synthetic examples by adding randin noise to each instance. Repeat until threshold is reached.
		synthetic_x, synthetic_y = [], []
		while len(synthetic_x) < cls.num_synthetic_examples:
			
			for y_true in all_instances:
				x_noised = []
				for idx, probability in enumerate(np.random.uniform(0.1, 1.0, y_true.shape[0])):
					if probability < 0.85:
						x_noised.append(y_true[idx])				
					else:
						x_noised.append(0 if y_true[idx] == 1 else 1)

				synthetic_x.append(np.array(x_noised, dtype=np.float32))
				synthetic_y.append(np.array(y_true, dtype=np.float32))

		return np.array(synthetic_x), np.array(synthetic_y)
	
	@classmethod
	def execute_training_data_gen(cls, package):

		objectives, x_train, y_train, x_validation, y_validation = package
		
		objectives.append(cls.primary_objective)

		problem = GenericOptimizer(
			x_train, 
			y_train, 
			x_validation, 
			y_validation,
			objectives,
			"Sequential"
		)

		algorithm = NSGA2(
			pop_size=GenericOptimizer.population_size, 
			sampling=BinaryRandomSampling(), 
			crossover=HUX(), 
			mutation=BitflipMutation(), 
			eliminate_duplicates=True
		)
		result = minimize(
			problem, 
			algorithm, 
			('n_gen', GenericOptimizer.population_size)
		)
		
		return result.X

folder = os.listdir('../Datasets')[0]
# def pipeline(folder):
x, y, splits = prepare_splits(os.path.join('../Datasets', folder, f"{folder}.csv"))

train_idx, test_idx, validation_idx = None, None, None
for train_idx, test_idx, validation_idx in splits:
	train_idx, test_idx, validation_idx = train_idx, test_idx, validation_idx
	break

x_train, y_train = x[train_idx], y[train_idx]
x_validation, y_validation = x[validation_idx], y[validation_idx]
x_test, y_test = x[test_idx], y[test_idx]

algorithm = NSGA2(
	pop_size=GenericOptimizer.population_size, 
	sampling=BinaryRandomSampling(), 
	crossover=HUX(), 
	mutation=CustomMutation(x_train, y_train, x_validation, y_validation), 
	eliminate_duplicates=True
)

problem = GenericOptimizer(
	x_train, 
	y_train, 
	x_validation, 
	y_validation,
	[CustomMutation.primary_objective, GenericOptimizer.calculate_num_examples],
	"Sequential"
)

result = minimize(
	problem, 
	algorithm, 
	('n_gen', GenericOptimizer.population_size)
)

validation_fitness = result.F
optimization_population = result.X

if len(optimization_population.shape) == 1:
	optimization_population = np.array([list(optimization_population)])

validation_pareto_front, validation_optimized_x_train, validation_optimized_y_train = GenericOptimizer.calculate_optimal_instance(
	x_train,
	y_train,
	x_validation,
	y_validation,
	validation_fitness,
	optimization_population,
	GenericOptimizer.n_neighbours
)

reduction_rate = 1 - (validation_optimized_x_train.shape[0] / x_train.shape[0])

optimized_test_err = GenericOptimizer.calculate_overall_error(
	validation_optimized_x_train,
	validation_optimized_y_train,
	x_test,
	y_test,
	GenericOptimizer.n_neighbours
)
optimized_validation_err = GenericOptimizer.calculate_overall_error(
	validation_optimized_x_train,
	validation_optimized_y_train,
	x_validation,
	y_validation,
	GenericOptimizer.n_neighbours
)
baseline_validation_err = GenericOptimizer.calculate_overall_error(
	x_train,
	y_train,
	x_validation,
	y_validation,
	GenericOptimizer.n_neighbours
)
baseline_test_err = GenericOptimizer.calculate_overall_error(
	x_train,
	y_train,
	x_test,
	y_test,
	GenericOptimizer.n_neighbours
)

test_fitness = []
for objective in [CustomMutation.primary_objective, GenericOptimizer.calculate_num_examples]:
	test_fitness.append(GenericOptimizer.unbound_eval_objectives(
		objective,
		optimization_population,
		x_train, y_train,
		x_test, y_test
	))
test_fitness = np.column_stack(test_fitness)

test_pareto_front, ideal_test_x_train, ideal_test_y_train = GenericOptimizer.calculate_optimal_instance(
	x_train,
	y_train,
	x_test,
	y_test,
	test_fitness,
	optimization_population,
	GenericOptimizer.n_neighbours
)

ideal_test_err = GenericOptimizer.calculate_overall_error(
	ideal_test_x_train,
	ideal_test_y_train,
	x_test,
	y_test,
	GenericOptimizer.n_neighbours
)

report = {
	"Baseline validation err": baseline_validation_err,
	"Baseline test err": baseline_test_err,
	"Optimzed validation err": optimized_validation_err,
	"Optimized test err": optimized_test_err,
	"Ideal test err": ideal_test_err,
	"result object": result
}
	# return report

# folder = os.listdir('../Datasets')[0]
# report = pipeline(folder)


C:\Users\RjKim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
C:\Users\RjKim\AppData\Local\Temp\ipykernel_38340\843797444.py:57: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  return np.array(predictions)


In [9]:
report

{'Baseline validation err': 0.36046511627906974,
 'Baseline test err': 0.3121387283236994,
 'Optimzed validation err': 0.10465116279069764,
 'Optimized test err': 0.3179190751445087,
 'Ideal test err': 0.3063583815028902,
 'result object': <pymoo.core.result.Result at 0x20e71ee5d90>}

In [7]:
len(report['result object'].X)

11

In [ ]:
for folder in os.listdir('../Datasets'):
	baseline_validation = []
	baseline_test = []
	optimized_test = []
	optimized_validation = []

	for _ in range(31):
		if os.path.exists(f'runs/{folder}_{_}.pickle'):
			print(f'passed on {folder}_{_}.pickle')
		
		else:
			try:
				print(f"Starting {folder} run {_}...")
					
				splits = step_1(folder)
				print(f"\tStep 1 complete...")

				primary_objective = GenericOptimizer.calculate_overall_error
				secondary_objectives = [
					[GenericOptimizer.calculate_num_examples],
					[GenericOptimizer.calculate_class0_error],
					[GenericOptimizer.calculate_class0_inverse_f1],
					[GenericOptimizer.calculate_class0_inverse_precision],
					[GenericOptimizer.calculate_class0_inverse_recall],
					[GenericOptimizer.calculate_class1_error],
					[GenericOptimizer.calculate_class1_inverse_f1],
					[GenericOptimizer.calculate_class1_inverse_precision],
					[GenericOptimizer.calculate_class1_inverse_recall],
					[GenericOptimizer.calculate_overall_inverse_f1],
					[GenericOptimizer.calculate_overall_inverse_precision],
					[GenericOptimizer.calculate_overall_inverse_recall],
					[GenericOptimizer.calculate_class0_inverse_precision, GenericOptimizer.calculate_class1_inverse_precision],
					[GenericOptimizer.calculate_class0_inverse_recall, GenericOptimizer.calculate_class1_inverse_recall],
					[GenericOptimizer.calculate_class0_inverse_f1, GenericOptimizer.calculate_class1_inverse_f1],
					[GenericOptimizer.calculate_class0_error, GenericOptimizer.calculate_class1_error],
				]
				populations = step_2(
					splits["X Train"], 
					splits["Y Train"], 
					splits["X Validation"], 
					splits["Y Validation"], 
					primary_objective, 
					secondary_objectives
				)
				print(f"\tStep 2 complete...")

				all_outcomes = step_3(populations)
				print(f"\tStep 3 complete...")

				x, y = step_4(all_outcomes)
				print(f"\tStep 4 complete...")

				model = step_5(x, y)
				print(f"\tStep 5 complete...")

				result = step_6(
					model, 
					splits["X Train"], 
					splits["Y Train"], 
					splits["X Validation"], 
					splits["Y Validation"], 
				)
				print(f"\tStep 6 complete...")

				baseline_validation_err, optimized_validation_err, baseline_test_err, optimized_test_err, reduction_rate = benchmark(
					result, 
					splits["X Train"], 
					splits["Y Train"], 
					splits["X Test"], 
					splits["Y Test"], 
					splits["X Validation"], 
					splits["Y Validation"], 
				)
				print(f"\tRun complete...\n")

				with open(f'runs/{folder}_{_}.pickle', 'wb') as fh:
					pickle.dump((baseline_test_err, baseline_validation_err, optimized_test_err, optimized_validation_err), fh)
			except Exception as e:
				print(f"Error: {e}")
				


passed on australian_0.pickle
passed on australian_1.pickle
passed on australian_2.pickle
passed on australian_3.pickle
passed on australian_4.pickle
passed on australian_5.pickle
passed on australian_6.pickle
passed on australian_7.pickle
passed on australian_8.pickle
passed on australian_9.pickle
passed on australian_10.pickle
passed on australian_11.pickle
passed on australian_12.pickle
passed on australian_13.pickle
passed on australian_14.pickle
passed on australian_15.pickle
passed on australian_16.pickle
passed on australian_17.pickle
passed on australian_18.pickle
passed on australian_19.pickle
passed on australian_20.pickle
passed on australian_21.pickle
passed on australian_22.pickle
passed on australian_23.pickle
passed on australian_24.pickle
passed on australian_25.pickle
passed on australian_26.pickle
passed on australian_27.pickle
passed on australian_28.pickle
passed on australian_29.pickle
passed on australian_30.pickle
passed on bupa_0.pickle
passed on bupa_1.pickle
p

C:\Users\RjKim\AppData\Local\Temp\ipykernel_21156\3556282856.py:33: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  return np.array(predictions)


	Step 6 complete...
	Run complete...

Starting phoneme run 1...
	Step 1 complete...
	Step 2 complete...
	Step 3 complete...
Error: Unable to allocate 5.04 GiB for an array with shape (500526, 2702) and data type float32
Starting phoneme run 2...
	Step 1 complete...
	Step 2 complete...
	Step 3 complete...
Error: Unable to allocate 5.04 GiB for an array with shape (500760, 2702) and data type float32
Starting phoneme run 3...
	Step 1 complete...
	Step 2 complete...
	Step 3 complete...
Error: Unable to allocate 5.04 GiB for an array with shape (500423, 2702) and data type float32
Starting phoneme run 4...
	Step 1 complete...
	Step 2 complete...
	Step 3 complete...
Error: Unable to allocate 5.03 GiB for an array with shape (500123, 2702) and data type float32
Starting phoneme run 5...
	Step 1 complete...
	Step 2 complete...
	Step 3 complete...
Error: Unable to allocate 5.04 GiB for an array with shape (500409, 2702) and data type float32
Starting phoneme run 6...
	Step 1 complete...
	Step 

C:\Users\RjKim\AppData\Local\Temp\ipykernel_21156\3556282856.py:33: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  return np.array(predictions)


	Step 6 complete...
	Run complete...

Starting pima run 1...
	Step 1 complete...
	Step 2 complete...
	Step 3 complete...
	Step 4 complete...
	Step 5 complete...


...

	Step 6 complete...
	Run complete...

Starting yeast1 run 29...
	Step 1 complete...
	Step 2 complete...
	Step 3 complete...
	Step 4 complete...
	Step 5 complete...


C:\Users\RjKim\AppData\Local\Temp\ipykernel_21156\3556282856.py:33: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  return np.array(predictions)


	Step 6 complete...
	Run complete...

Starting yeast1 run 30...
	Step 1 complete...
	Step 2 complete...
	Step 3 complete...
	Step 4 complete...
	Step 5 complete...


C:\Users\RjKim\AppData\Local\Temp\ipykernel_21156\3556282856.py:33: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  return np.array(predictions)


	Step 6 complete...
	Run complete...

